In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
import pandas as pd

In [3]:
import dataframe_image as dfi

In [4]:
from plot_config import *
from utils import *

In [5]:
rerun = False

In [6]:
if rerun:
    final = []
    
    for case in ['fresh','salty']:
        print(case)
        map,table = open_case(case)
        
        for duration in [1,2,5,10]:
            print(duration)
            this_sum = 0
            for var in rhs_list:
                print(var, end = ' ')
                contr = float(table[var].isel(time = slice(0,int(duration*365))).sum(dim = 'time').mean(dim = 'space'))
                final.append([case,duration,var,contr])
                this_sum+= contr
            print()
    
            upper = np.minimum(len(table.time)-1,int(duration*365))
            init = float(table['sf'][upper].mean(dim = 'space'))
            endd = float(table['sl'][0].mean(dim = 'space'))
            final.append([case,duration,'init',init])
            final.append([case,duration,'Error',endd-init+this_sum])
    
    final = np.array(final)
    
    np.save('lineplotprep/table1.npy',final)

In [7]:
final = np.load('lineplotprep/table1.npy')
# final

In [8]:
df = pd.DataFrame(final, columns = ['Event','Duration (yr)','Term','Contribution'])
df

,Event,Duration (yr),Term,Contribution
0,fresh,1,e_ua,-0.0003364145955357353
1,fresh,1,E,0.02967890017978906
2,fresh,1,dif_h,-0.03261566123834041
3,fresh,1,dif_v,-0.07232158664940097
4,fresh,1,A,0.08206349941647725
...,...,...,...,...
67,salty,10,A,-0.09671983199997847
68,salty,10,I,-0.0122479541028835
69,salty,10,F,-0.035440690533642134
70,salty,10,init,0.006082533558050084


In [9]:
event_dic = {
    'fresh':'June, 2016',
    'salty':'Jan., 2007'
}
t_dic = {
    'init': 'Initial Anomaly',
    'Error':'Error'
}
t_dic.update(case_term_dic)
df['Event'] = [event_dic[i] for i in df['Event']]
df['Term'] = [t_dic[i] for i in df['Term']]
df['Contribution'] = [-float(i) for i in df['Contribution']]

In [10]:
correct_order = [t_dic[i] for i in rhs_list+['init','Error']]

In [11]:
df = df.set_index('Term').pivot(columns = ['Event', 'Duration (yr)'],values='Contribution').reindex(correct_order)

In [12]:
vmax = 0.2
def to2f(num):
    return f'{num:.3f}'
def make_pretty(styler):
    styler.format(to2f)
    styler.background_gradient(axis=None, vmin=-vmax, vmax=vmax, cmap=balance)
    return styler
styled = df.style.pipe(make_pretty)
styled

In [13]:
dfi.export(styled, 'Table1.png')

OSError: Chrome executable not able to be found on your machine